In [42]:
# Prices are in €/MWh while production figures are in kWh it is not handled yet

import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB

prices_df = pd.read_csv('../Data assignment 1/prices_merged_df_output.csv', delimiter=',')

features_df = pd.read_csv('../Data assignment 1/Feature data.csv', delimiter=',')


features_df['AKI Kalby Active Power'] = -1 * features_df['AKI Kalby Active Power']

#2022-06-08 01:00:00,295.754578,168.969971,165.1
data = prices_df
data['actProd'] = features_df['AKI Kalby Active Power']
data["HourDK"] = pd.to_datetime(data["HourDK"])
Start_Observation = pd.to_datetime("2022-01-01 00:00:00")
End_Observation = pd.to_datetime("2022-01-01 01:00:00")
condition = (data["HourDK"]>= Start_Observation) & (data["HourDK"]< End_Observation)

data  = data[condition]

capacity = 30000.0

p_t = np.random.uniform(0.0, capacity, size=(1, 1))
p_t = np.round(p_t, 2)
data['modProd'] = p_t

print(data)
solutions  = list()
for index,row in data.iterrows():
    
        dailyRev = row['DA_PriceEUR'] * row['modProd']
        
        if row['modProd'] > row['actProd']:
            dailyRev = dailyRev - row['BalancingPriceUpEUR'] * (row['modProd']-row['actProd'])
        elif row['actProd'] > row['modProd']:
            dailyRev = dailyRev + row['BalancingPriceDownEUR'] * (row['actProd']-row['modProd'])
            
        solutions.append({row['HourDK']:dailyRev})






# p_t_hat as decision variable
def OptimizationProblemEnergybid():
    model = gp.Model()
    solutions  = list()
    
    for index, row in prices_df.iterrows():
        
        p_t_hat = model.addVar(name="p_t_hat", lb=0, ub=capacity, vtype=GRB.INTEGER)
        z_down = model.addVar(name="z_down", vtype=GRB.CONTINUOUS, lb=0)
        z_up = model.addVar(name="z_up", vtype=GRB.CONTINUOUS, lb=0)
        
        condition_down = model.addVar(name="condition_down", vtype=GRB.BINARY)
        condition_up = model.addVar(name="condition_up", vtype=GRB.BINARY)
        
        model.addConstr(z_up <= (p_t_hat - row['p_t']) * condition_up)
        model.addConstr(z_up >= (p_t_hat - row['p_t']) * condition_up)
        model.addConstr(z_down <= (row['p_t'] - p_t_hat) * condition_down)
        model.addConstr(z_down >= (row['p_t'] - p_t_hat) * condition_down)
        model.addConstr(condition_up + condition_down == 1)
        
    
        #defining the constraints
        model.addConstr(-p_t_hat <= 0)
        model.addConstr(p_t_hat <= capacity)
        model.addConstr(-z_down <= 0)
        model.addConstr(-z_up <= 0)
        
        model.setObjective((row['DA_PriceEUR']*p_t_hat)+(row['BalancingPriceDownEUR']*z_down-row['BalancingPriceUpEUR']*z_up), GRB.MAXIMIZE)
        
        model.optimize()
    
        if model.status == GRB.OPTIMAL:
            print("Optimal solution found!")
            print(f"Objective Value: {model.ObjVal}")
            values = []
            values.append({ "Day-Ahead-Price": row['DA_PriceEUR'],
                            "BalancingPriceUp":row['BalancingPriceUpEUR'],
                            "BalancingPriceDown":row['BalancingPriceDownEUR'],
                            "p_t":row['p_t'],
                            "Revenue":model.ObjVal})
            for v in model.getVars():
                print(f"{v.varName}: {v.x}")
                values.append({v.varName:v.x})
        
            
            solutions.append({row['HourDK']:values})
                
        elif model.status == GRB.INFEASIBLE:
            print("Model is infeasible.")
            model.computeIIS()
            model.write("infeasible.ilp")  # Write IIS to a file for review
            for c in model.getConstrs():
                if c.IISConstr:
                    print(f"Infeasible constraint: {c.constrName}")
            
            
        elif model.status == GRB.UNBOUNDED:
            print("Model is unbounded.")
        else:
            print("Model status:", model.status)
            
        model.remove(model.getConstrs())
        model.remove(model.getVars())
        
        
def QualityAssesmentModellEnergyRevenue(Start_Observation, End_Observation, data):
    pass

      HourDK  BalancingPriceUpEUR  BalancingPriceDownEUR  DA_PriceEUR  \
0 2022-01-01             46.59988               46.59988         46.6   

       actProd  modProd  
0  1893.541825  8237.53  


/var/folders/5y/2d5f2m356xd_kxc8krzn20q40000gn/T/ipykernel_11508/3584404752.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['modProd'] = p_t


In [43]:
solutions

[{Timestamp('2022-01-01 00:00:00'): 88239.81032329769}]